Definomos un Crawler

In [1]:
# Instalamos las librerías necesarias
%pip install scrapy bs4 elasticsearch

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from crawler import BulbaSpyder
import scrapy, os
from scrapy.crawler import CrawlerProcess

# Creamos un proceso de Crawler podemos poner distintas settings que están definidas en la documentación.
# Entre ellas podemos ocular los logs del proceso de Crawling.
process = CrawlerProcess(settings={
    "LOG_ENABLED": False,
    # Used for pipeline 1
})

# Comprobamos que existe la carpeta y si no existe la creamos
if not os.path.exists('bulbapedia'):
    os.mkdir('bulbapedia')

# Creamos el proceso con el RSSSpider
process.crawl(BulbaSpyder)
# Ejecutamos el Crawler
process.start()

ELEMENTO:
{
    "url": "https://bulbapedia.bulbagarden.net/wiki/Abomasnow_(Pok%C3%A9mon)",
    "pokemon": "Abomasnow",
    "numero_pokedex": "0460",
    "descripcion": "Abomasnow is a large, bipedal Pokémon covered in shaggy, white fur. It has pale purple eyes with bushy eyebrows and long strands of fur covering its mouth. Two tufts of longer fur grow on both its back and chest, with the chest fur being longer on the female. Its hands, feet, and tail are dark green and spiky, similar to the foliage of an evergreen tree. On its back are four green spikes that resemble pinecones. Abomasnow lives in snowy mountains and appears only when snow flowers bloom. Once the flowers die, it retreats to isolated areas because Abomasnow prefers to be alone and not associate with others. It is able to create blizzards to hide itself and can also cause whiteout conditions with just a shake of its massive body. Abomasnow is also known as the \"Ice Monster\" and \"abominable snowman\". It protects Snover